In [2]:
import pandas as pd
import numpy as np
import requests

# Tipos de Datos Necesarios
Datos de Características del Juego

## Qué: Información sobre las características internas del videojuego.
- Ejemplo: Género, plataforma, desarrollador, editor, fecha de lanzamiento, clasificación por edad, modo de juego (single-player, multiplayer), gráficos (2D/3D), historia, tipo de monetización (compras dentro del juego, DLCs).

Dónde Obtenerlo:
- APIs: IGDB API, RAWG API.
- Scraping: Sitios como Metacritic o Steam para características de juegos.
- Bases de Datos Públicas: Vgchartz para datos de juegos pasados.
- Datos Históricos de Ventas e Ingresos

## Qué: Historial de ventas y desempeño comercial de videojuegos anteriores.
- Ejemplo: Ventas en la primera semana, ventas totales, ingresos generados, número de copias vendidas por región.

Dónde Obtenerlo:

- APIs: SteamSpy API (para datos de juegos en Steam).
- Scraping: Vgchartz, sitios de análisis de la industria.
- Bases de Datos Públicas: Reportes de la industria, como SuperData (algunas versiones son gratuitas).
- Datos de Marketing y Publicidad

## Qué: Información sobre las campañas de marketing y estrategias de lanzamiento.
- Ejemplo: Presupuesto de marketing, número de trailers lanzados, eventos en los que se presentó el juego, menciones en redes sociales, engagement en plataformas como YouTube y Twitch.

Dónde Obtenerlo:

- APIs: YouTube API, Twitch API.
- Scraping: Redes sociales, YouTube, Twitch para obtener datos de campañas de marketing.
- Datos Públicos: Análisis de marketing de empresas de videojuegos, informes de inversión publicitaria.
- Datos de Tendencias del Mercado

## Qué: Datos sobre tendencias actuales y futuras en la industria de videojuegos.
- Ejemplo: Popularidad de ciertos géneros o plataformas, tendencias en modelos de monetización, impacto de influencers en la industria.

Dónde Obtenerlo:

- APIs: Google Trends API, Twitter API.
- Scraping: Foros de videojuegos como Reddit, análisis en sitios como Polygon o Kotaku.
- Datos Públicos: Informes de la industria como los de Newzoo.
- Datos de Opiniones y Críticas Previas

## Qué: Evaluaciones anticipadas del juego antes de su lanzamiento.
- Ejemplo: Expectativas de críticos y jugadores, anticipación en foros y redes sociales, evaluaciones tempranas (previews, demos).

Dónde Obtenerlo:

- APIs: Metacritic API (aunque está más limitado), o scraping.
- Scraping: Foros de discusión como Reddit, NeoGAF, y análisis en sitios de reseñas.

Api key = 52b89ce33c384c0fb7199d5c636ed6f9

# API

In [97]:
import requests
import pandas as pd

api_key = 'f36b1ea3e7a64c838d7514f00104d1e5'

url_games = f'https://api.rawg.io/api/games?key={api_key}'

response_games = requests.get(url_games)

if response_games.status_code == 200:
    response_json = response_games.json()
    games_data = response_json.get('results', []) 
    df_games = pd.DataFrame(games_data)
else:
    print(f"Error: {response_games.status_code}")
    response_games.text


In [98]:
api_key = 'f36b1ea3e7a64c838d7514f00104d1e5'
url_platforms = f'https://api.rawg.io/api/games?key={api_key}'

response_platforms = requests.get(url_platforms)

if response_platforms.status_code == 200:
    platforms_data = response_platforms.json()['results']
    ps5_id = next((platform['id'] for platform in platforms_data if platform['name'] == 'PlayStation 5'), None)
    if ps5_id:
        print(f"PlayStation 5 ID: {ps5_id}")
    else:
        print("PlayStation 5 no encontrada")
else:
    print(f"Error: {response_platforms.status_code}")
    response_platforms.text


PlayStation 5 no encontrada


In [5]:
import requests
import pandas as pd

In [7]:
api_key = '83ebefc6c8954a95a4079d02ad293e58'

all_games = []
page = 1
page_size = 20

while True:
    url_games = 'https://api.rawg.io/api/games'
    params = {
        'key': api_key,
        'page': page,
        'page_size': page_size
    }

    response_games = requests.get(url_games, params=params)
    
    if response_games.status_code == 200:
        response_json = response_games.json()
        games_data = response_json.get('results', [])
        
        if not games_data:
            print(f"No hay más juegos en la página {page}.")
            break
        
        all_games.extend(games_data)
        
        page += 1
    elif response_games.status_code == 404:
        print(f"Error 404: Página {page} no encontrada. Puede que se haya alcanzado el final de los resultados.")
        break
    else:
        print(f"Error: {response_games.status_code}")
        print(response_games.text)
        break

df_games = pd.DataFrame(all_games)

df_games.head()
print(f"Total de juegos obtenidos: {len(df_games)}")


SSLError: HTTPSConnectionPool(host='api.rawg.io', port=443): Max retries exceeded with url: /api/games?key=83ebefc6c8954a95a4079d02ad293e58&page=2977&page_size=20 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))

In [8]:
import time
import requests
import pandas as pd

# Definir la clave de API y los parámetros iniciales
api_key = '83ebefc6c8954a95a4079d02ad293e58'
all_games = []
page = 1
page_size = 20

# Función para realizar peticiones con reintentos y backoff exponencial
def fetch_data_with_retry(url, params, retries=5):
    delay = 1  # Tiempo de espera inicial en segundos
    for i in range(retries):
        try:
            response = requests.get(url, params=params, timeout=10)  # Timeout de 10 segundos
            response.raise_for_status()  # Levanta un error si la respuesta no es 200
            return response
        except requests.exceptions.RequestException as e:
            print(f"Intento {i + 1} fallido: {e}")
            time.sleep(delay)
            delay *= 2  # Aumenta el tiempo de espera exponencialmente
    print("No se pudo obtener la respuesta después de varios intentos.")
    return None

# Bucle principal para obtener los juegos
while True:
    url_games = 'https://api.rawg.io/api/games'
    params = {
        'key': api_key,
        'page': page,
        'page_size': page_size
    }

    response_games = fetch_data_with_retry(url_games, params)  # Usa la función con reintento
    
    if response_games and response_games.status_code == 200:
        response_json = response_games.json()
        games_data = response_json.get('results', [])
        
        if not games_data:
            print(f"No hay más juegos en la página {page}.")
            break
        
        all_games.extend(games_data)
        
        page += 1
    elif response_games and response_games.status_code == 404:
        print(f"Error 404: Página {page} no encontrada. Puede que se haya alcanzado el final de los resultados.")
        break
    else:
        print(f"Error: {response_games.status_code if response_games else 'Sin respuesta'}")
        if response_games:
            print(response_games.text)
        break

# Convertir los juegos obtenidos a un DataFrame
df_games = pd.DataFrame(all_games)

# Mostrar los primeros registros y el total de juegos obtenidos
df_games.head()
print(f"Total de juegos obtenidos: {len(df_games)}")


Intento 1 fallido: 502 Server Error: Bad Gateway for url: https://api.rawg.io/api/games?key=83ebefc6c8954a95a4079d02ad293e58&page=17&page_size=20
Intento 2 fallido: 502 Server Error: Bad Gateway for url: https://api.rawg.io/api/games?key=83ebefc6c8954a95a4079d02ad293e58&page=17&page_size=20
Intento 1 fallido: 502 Server Error: Bad Gateway for url: https://api.rawg.io/api/games?key=83ebefc6c8954a95a4079d02ad293e58&page=22&page_size=20
Intento 1 fallido: 502 Server Error: Bad Gateway for url: https://api.rawg.io/api/games?key=83ebefc6c8954a95a4079d02ad293e58&page=25&page_size=20
Intento 1 fallido: 502 Server Error: Bad Gateway for url: https://api.rawg.io/api/games?key=83ebefc6c8954a95a4079d02ad293e58&page=27&page_size=20
Intento 1 fallido: 502 Server Error: Bad Gateway for url: https://api.rawg.io/api/games?key=83ebefc6c8954a95a4079d02ad293e58&page=28&page_size=20
Intento 1 fallido: HTTPSConnectionPool(host='api.rawg.io', port=443): Read timed out. (read timeout=10)
Intento 1 fallido: 4

In [9]:
df_games.to_csv('ws2_rawg1.csv',index=False)

In [358]:
df = pd.read_csv('/Users/Jaimecalderon/Desktop/IRONHACK/Proyectos/Proyecto final/Videojuegos/ws2_rawg1.csv')

In [359]:
import ast
df['platforms'] = df['platforms'].apply(ast.literal_eval)

In [306]:
df['platforms'][0]

[{'platform': {'id': 4,
   'name': 'PC',
   'slug': 'pc',
   'image': None,
   'year_end': None,
   'year_start': None,
   'games_count': 535023,
   'image_background': 'https://media.rawg.io/media/games/511/5118aff5091cb3efec399c808f8c598f.jpg'},
  'released_at': '2013-09-17',
  'requirements_en': {'minimum': 'Minimum:OS: Windows 10 64 Bit, Windows 8.1 64 Bit, Windows 8 64 Bit, Windows 7 64 Bit Service Pack 1, Windows Vista 64 Bit Service Pack 2* (*NVIDIA video card recommended if running Vista OS)Processor: Intel Core 2 Quad CPU Q6600 @ 2.40GHz (4 CPUs) / AMD Phenom 9850 Quad-Core Processor (4 CPUs) @ 2.5GHzMemory: 4 GB RAMGraphics: NVIDIA 9800 GT 1GB / AMD HD 4870 1GB (DX 10, 10.1, 11)Storage: 72 GB available spaceSound Card: 100% DirectX 10 compatibleAdditional Notes: Over time downloadable content and programming changes will change the system requirements for this game.  Please refer to your hardware manufacturer and www.rockstargames.com/support for current compatibility informa

In [360]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [75]:
df_games.dtypes

id                 int64
slug              object
name              object
released          object
tba                 bool
                   ...  
App Store           bool
Xbox Store          bool
Steam               bool
Epic Games          bool
Xbox 360 Store      bool
Length: 93, dtype: object

In [72]:
df.dtypes

id                      int64
slug                   object
name                   object
released               object
tba                      bool
background_image       object
rating                float64
rating_top              int64
ratings                object
ratings_count           int64
reviews_text_count      int64
added                   int64
added_by_status        object
metacritic            float64
playtime                int64
suggestions_count       int64
updated                object
user_game             float64
reviews_count           int64
saturated_color        object
dominant_color         object
platforms              object
parent_platforms       object
genres                 object
stores                 object
clip                  float64
tags                   object
esrb_rating            object
short_screenshots      object
community_rating      float64
platform_names         object
store_names            object
dtype: object

In [147]:
df["platforms"][0][2]

{'platform': {'id': 186,
  'name': 'Xbox Series S/X',
  'slug': 'xbox-series-x',
  'image': None,
  'year_end': None,
  'year_start': 2020,
  'games_count': 972,
  'image_background': 'https://media.rawg.io/media/games/ffe/ffed87105b14f5beff72ff44a7793fd5.jpg'},
 'released_at': '2013-09-17',
 'requirements_en': None,
 'requirements_ru': None}

In [361]:
def extract_platform_names(platforms):
    if not isinstance(platforms, list):
        return []
    return [platform['platform']['name'] for platform in platforms]

def extract_games_count(platforms):
    if not isinstance(platforms, list):
        return []
    return [platform['platform']['games_count'] for platform in platforms]

df['platform_names'] = df['platforms'].apply(extract_platform_names)
df['games_counts'] = df['platforms'].apply(extract_games_count)

df[['name', 'platform_names', 'games_counts']]


,name,platform_names,games_counts
0,Grand Theft Auto V,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]"
1,The Witcher 3: Wild Hunt,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]"
2,Portal 2,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]"
3,Counter-Strike: Global Offensive,"[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]"
4,Tomb Raider (2013),"[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]"
...,...,...,...
31075,Tiny Life,"[Linux, macOS, PC]","[77481, 104591, 535135]"
31076,Dungeon Drafters,"[PC, Linux]","[535135, 77481]"
31077,Havendock,[PC],[535135]
31078,Apewar,[PC],[535135]


In [104]:
df_games['game_id'] = df_games['id']
df_games.drop(columns='id',inplace=True)

In [105]:
df_games

slug  \
0                                     grand-theft-auto-v   
1                                the-witcher-3-wild-hunt   
2                             the-elder-scrolls-v-skyrim   
3                                              destiny-2   
4                                              fallout-4   
...                                                  ...   
1103                   dead-by-daylight-spark-of-madness   
1104  pre-purchase-romancing-saga-minstrel-song-remaster   
1105                              the-courier-pm-studios   
1106                                        neo-breakout   
1107                                          steel-seed   

                                                         name  playtime  \
0                                          Grand Theft Auto V        74   
1                                    The Witcher 3: Wild Hunt        45   
2                                 The Elder Scrolls V: Skyrim        47   
3                                                   Destiny 2         5   
4                                                   Fallout 4        40   
...                                                       ...       ...   
1103                      Dead by Daylight - Spark of Madness         0   
1104  Pre-purchase Romancing SaGa - Minstrel Song- Remastered         0   
1105                                 The Courier [PM Studios]         0   
1106                                             Neo Breakout         0   
1107                                               Steel Seed         0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    platforms  \
0                                                                                    [{'platform': {'id': 4, 'name': 'PC', 'slug': 'pc'}}, {'platform': {'id': 187, 'name': 'PlayStation 5', 'slug': 'playstation5'}}, {'platform': {'id': 1, 'name': 'Xbox One', 'slug': 'xbox-one'}}, {'platform': {'id': 18, 'name': 'PlayStation 4', 'slug': 'playstation4'}}, {'platform': {'id': 186, 'name': 'Xbox Series S/X', 'slug': 'xbox-series-x'}}, {'platform': {'id': 14, 'name': 'Xbox 360', 'slug': 'xbox360'}}, {'platform': {'id': 16, 'name': 'PlayStation 3', 'slug': 'playstation3'}}]   
1                                                                                      [{'platform': {'id': 4, 'name': 'PC', 'slug': 'pc'}}, {'platform': {'id': 187, 'name': 'PlayStation 5', 'slug': 'playstation5'}}, {'platform': {'id': 1, 'name': 'Xbox One', 'slug': 'xbox-one'}}, {'platform': {'id': 18, 'name': 'PlayStation 4', 'slug': 'playstation4'}}, {'platform': {'id': 186, 'name': 'Xbox Series S/X', 'slug': 'xbox-series-x'}}, {'platform': {'id': 7, 'name': 'Nintendo Switch', 'slug': 'nintendo-switch'}}, {'platform': {'id': 5, 'name': 'macOS', 'slug': 'macos'}}]   
2     [{'platform': {'id': 4, 'name': 'PC', 'slug': 'pc'}}, {'platform': {'id': 187, 'name': 'PlayStation 5', 'slug': 'playstation5'}}, {'platform': {'id': 1, 'name': 'Xbox One', 'slug': 'xbox-one'}}, {'platform': {'id': 18, 'name': 'PlayStation 4', 'slug': 'playstation4'}}, {'platform': {'id': 186, 'name': 'Xbox Series S/X', 'slug': 'xbox-series-x'}}, {'platform': {'id': 7, 'name': 'Nintendo Switch', 'slug': 'nintendo-switch'}}, {'platform': {'id': 14, 'name': 'Xbox 360', 'slug': 'xbox360'}}, {'platform': {'id': 16, 'name': 'PlayStation 3', 'slug': 'playstation3'}}]   
3                                                                                                                         

In [362]:
df['stores'] = df['stores'].apply(ast.literal_eval)

In [151]:
df['stores'][0][0]

{'id': 290375,
 'store': {'id': 3,
  'name': 'PlayStation Store',
  'slug': 'playstation-store',
  'domain': 'store.playstation.com',
  'games_count': 7961,
  'image_background': 'https://media.rawg.io/media/games/490/49016e06ae2103881ff6373248843069.jpg'}}

In [363]:
def extract_platform_names(stores):
    if not isinstance(stores, list):
        return []
    return [store['store']['name'] for store in stores]

def extract_games_count(stores):
    if not isinstance(stores, list):
        return []
    return [store['store']['games_count'] for store in stores]

df['store_names'] = df['stores'].apply(extract_platform_names)
df['games_counts_stores'] = df['stores'].apply(extract_games_count)

df[['name', 'store_names', 'games_counts_stores']]


,name,store_names,games_counts_stores
0,Grand Theft Auto V,"[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]"
1,The Witcher 3: Wild Hunt,"[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]"
2,Portal 2,"[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]"
3,Counter-Strike: Global Offensive,"[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]"
4,Tomb Raider (2013),"[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]"
...,...,...,...
31075,Tiny Life,[Steam],[98197]
31076,Dungeon Drafters,"[GOG, Steam]","[6290, 98197]"
31077,Havendock,[Steam],[98197]
31078,Apewar,[Steam],[98197]


In [107]:
df_games.drop(columns='stores',inplace=True)

In [108]:
df_games['genres'][0]

[{'id': 4, 'name': 'Action', 'slug': 'action'}]

In [364]:
df['genres'] = df['genres'].apply(ast.literal_eval)

In [161]:
df['genres']

0        [{'id': 4, 'name': 'Action', 'slug': 'action',...
1        [{'id': 4, 'name': 'Action', 'slug': 'action',...
2        [{'id': 2, 'name': 'Shooter', 'slug': 'shooter...
3        [{'id': 2, 'name': 'Shooter', 'slug': 'shooter...
4        [{'id': 4, 'name': 'Action', 'slug': 'action',...
                               ...                        
31075    [{'id': 14, 'name': 'Simulation', 'slug': 'sim...
31076    [{'id': 5, 'name': 'RPG', 'slug': 'role-playin...
31077    [{'id': 10, 'name': 'Strategy', 'slug': 'strat...
31078    [{'id': 4, 'name': 'Action', 'slug': 'action',...
31079    [{'id': 3, 'name': 'Adventure', 'slug': 'adven...
Name: genres, Length: 31080, dtype: object

In [365]:
def extract_platform_names(genres):
    if not isinstance(genres, list):
        return []
    return [genre['name'] for genre in genres]

def extract_games_count(genres):
    if not isinstance(genres, list):
        return []
    return [genre['games_count'] for genre in genres]

df['genre_names'] = df['genres'].apply(extract_platform_names)
df['games_counts_genres'] = df['genres'].apply(extract_games_count)

df[['name', 'genre_names', 'games_counts_genres']]


,name,genre_names,games_counts_genres
0,Grand Theft Auto V,[Action],[181902]
1,The Witcher 3: Wild Hunt,"[Action, RPG]","[181902, 56940]"
2,Portal 2,"[Shooter, Puzzle]","[59498, 97296]"
3,Counter-Strike: Global Offensive,[Shooter],[59498]
4,Tomb Raider (2013),"[Action, Fighting]","[181902, 11756]"
...,...,...,...
31075,Tiny Life,"[Simulation, Casual, Indie]","[70713, 55620, 68617]"
31076,Dungeon Drafters,"[RPG, Strategy, Indie]","[56960, 57235, 68617]"
31077,Havendock,"[Strategy, Simulation, Casual, Indie]","[57235, 70713, 55620, 68617]"
31078,Apewar,"[Action, Strategy, Casual, Indie]","[181942, 57235, 55620, 68617]"


In [19]:
df_games

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,...,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating,genre_names,store_names,platform_names
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,...,"[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290375, 'store': {'id': 3, 'name': 'Pl...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,[Action],"[PlayStation Store, Epic Games, Steam, Xbox 36...","[PC, PlayStation 5, Xbox Series S/X, PlayStati..."
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,...,"[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[Action, RPG]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[Xbox Series S/X, PlayStation 5, macOS, PlaySt..."
2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,...,"[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[Shooter, Puzzle]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb..."
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,...,"[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...",None,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,[Shooter],"[PlayStation Store, Steam, Xbox 360 Store]","[PC, Linux, Xbox 360, PlayStation 3]"
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,...,"[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 33824, 'store': {'id': 7, 'name': 'Xbo...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[Action, Fighting]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[PlayStation 4, PlayStation 3, Xbox 360, macOS..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31075,960362,tiny-life-2,Tiny Life,2023-05-03,False,https://media.rawg.io/media/screenshots/b8d/b8...,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",2,...,"[{'id': 14, 'name': 'Simulation', 'slug': 'sim...","[{'id': 959201, 'store': {'id': 1, 'name': 'St...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0,"[Simulation, Casual, Indie]",[Steam],"[Linux, macOS, PC]"
31076,960061,dungeon-drafters-2,Dungeon Drafters,2023-04-26,False,https://media.rawg.io/media/screenshots/53c/53...,0.00,0,[],0,...,"[{'id': 5, 'name': 'RPG', 'slug': 'role-playin...","[{'id': 958865, 'store': {'id': 5, 'name': 'GO...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0,"[RPG, Strategy, Indie]","[GOG, Steam]","[PC, Linux]"
31077,959743,havendock,Havendock,None,Fals

In [110]:
df_games.drop(columns='genres', inplace=True)

In [111]:
df_games.drop(columns='slug', inplace= True)

In [112]:
df_games.columns

Index(['name', 'playtime', 'platforms', 'released', 'tba', 'background_image',
       'rating', 'rating_top', 'ratings', 'ratings_count',
       'reviews_text_count', 'added', 'added_by_status', 'metacritic',
       'suggestions_count', 'updated', 'score', 'clip', 'tags', 'esrb_rating',
       'user_game', 'reviews_count', 'saturated_color', 'dominant_color',
       'short_screenshots', 'parent_platforms', 'community_rating',
       'platform_names', 'game_id', 'store_names', 'genre_names'],
      dtype='object')

In [113]:
df_games.drop(columns=['background_image', 'short_screenshots'], inplace= True)

In [114]:
df_games['platforms']

0                                                                                      [{'platform': {'id': 4, 'name': 'PC', 'slug': 'pc'}}, {'platform': {'id': 187, 'name': 'PlayStation 5', 'slug': 'playstation5'}}, {'platform': {'id': 1, 'name': 'Xbox One', 'slug': 'xbox-one'}}, {'platform': {'id': 18, 'name': 'PlayStation 4', 'slug': 'playstation4'}}, {'platform': {'id': 186, 'name': 'Xbox Series S/X', 'slug': 'xbox-series-x'}}, {'platform': {'id': 14, 'name': 'Xbox 360', 'slug': 'xbox360'}}, {'platform': {'id': 16, 'name': 'PlayStation 3', 'slug': 'playstation3'}}]
1                                                                                        [{'platform': {'id': 4, 'name': 'PC', 'slug': 'pc'}}, {'platform': {'id': 187, 'name': 'PlayStation 5', 'slug': 'playstation5'}}, {'platform': {'id': 1, 'name': 'Xbox One', 'slug': 'xbox-one'}}, {'platform': {'id': 18, 'name': 'PlayStation 4', 'slug': 'playstation4'}}, {'platform': {'id': 186, 'name': 'Xbox Series S/X', 'slug': 'xbox

In [224]:
df

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,reviews_text_count,added,added_by_status,metacritic,playtime,suggestions_count,updated,user_game,reviews_count,saturated_color,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,"{'yet': 532, 'owned': 12160, 'beaten': 5978, '...",92.0,74,434,2024-08-29T07:36:57,NaN,6964,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290375, 'store': {'id': 3, 'name': 'Pl...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902]
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,"{'yet': 1123, 'owned': 12013, 'beaten': 4855, ...",92.0,45,677,2024-08-28T20:12:31,NaN,6715,0f0f0f,0f0f0f,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]"
2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,"{'yet': 625, 'owned': 12214, 'beaten': 5519, '...",95.0,11,553,2024-08-29T07:23:05,NaN,5737,0f0f0f,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]"
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,"{'yet': 262, 'owned': 13146, 'beaten': 1048, '...",81.0,64,589,2024-08-20T10:48:54,NaN,3515,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498]
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 

In [313]:
all_platforms = set(platform for sublist in df['platform_names'] for platform in sublist)
all_platforms

{'3DO',
 'Android',
 'Apple II',
 'Atari 2600',
 'Atari 5200',
 'Atari 7800',
 'Atari 8-bit',
 'Atari Flashback',
 'Atari Lynx',
 'Atari ST',
 'Atari XEGS',
 'Classic Macintosh',
 'Commodore / Amiga',
 'Dreamcast',
 'Game Boy',
 'Game Boy Advance',
 'Game Boy Color',
 'Game Gear',
 'GameCube',
 'Genesis',
 'Jaguar',
 'Linux',
 'NES',
 'Neo Geo',
 'Nintendo 3DS',
 'Nintendo 64',
 'Nintendo DS',
 'Nintendo DSi',
 'Nintendo Switch',
 'PC',
 'PS Vita',
 'PSP',
 'PlayStation',
 'PlayStation 2',
 'PlayStation 3',
 'PlayStation 4',
 'PlayStation 5',
 'SEGA 32X',
 'SEGA CD',
 'SEGA Master System',
 'SEGA Saturn',
 'SNES',
 'Web',
 'Wii',
 'Wii U',
 'Xbox',
 'Xbox 360',
 'Xbox One',
 'Xbox Series S/X',
 'iOS',
 'macOS'}

In [116]:
df_games['store_names']

0       [Steam, PlayStation Store, Xbox Store, Xbox 360 Store, Epic Games]
1              [Steam, PlayStation Store, Xbox Store, GOG, Nintendo Store]
2               [Steam, PlayStation Store, Nintendo Store, Xbox 360 Store]
3                       [Steam, PlayStation Store, Xbox Store, Epic Games]
4                                   [Steam, PlayStation Store, Xbox Store]
                                       ...                                
1103                                                               [Steam]
1104                                               [Steam, Nintendo Store]
1105                                                                    []
1106                                                                    []
1107                                                                    []
Name: store_names, Length: 1108, dtype: object

In [314]:
all_stores = set(store for sublist in df['store_names'] for store in sublist)
all_stores

{'App Store',
 'Epic Games',
 'GOG',
 'Google Play',
 'Nintendo Store',
 'PlayStation Store',
 'Steam',
 'Xbox 360 Store',
 'Xbox Store',
 'itch.io'}

In [119]:
for platform in all_platforms:
    df[platform] = df['platform_names'].apply(lambda x: platform in x)
df.drop(columns='platforms', inplace= True)

In [131]:
df

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,reviews_text_count,added,added_by_status,metacritic,playtime,suggestions_count,updated,user_game,reviews_count,saturated_color,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating,platform_names,store_names,genre_names
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,"{'yet': 532, 'owned': 12160, 'beaten': 5978, '...",92.0,74,434,2024-08-29T07:36:57,NaN,6964,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290375, 'store': {'id': 3, 'name': 'Pl...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[PlayStation Store, Epic Games, Steam, Xbox 36...",[Action]
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,"{'yet': 1123, 'owned': 12013, 'beaten': 4855, ...",92.0,45,677,2024-08-28T20:12:31,NaN,6715,0f0f0f,0f0f0f,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[Action, RPG]"
2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,"{'yet': 625, 'owned': 12214, 'beaten': 5519, '...",95.0,11,553,2024-08-29T07:23:05,NaN,5737,0f0f0f,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[Shooter, Puzzle]"
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,"{'yet': 262, 'owned': 13146, 'beaten': 1048, '...",81.0,64,589,2024-08-20T10:48:54,NaN,3515,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PC, Linux, Xbox 360, PlayStation 3]","[PlayStation Store, Steam, Xbox 360 Store]",[Shooter]
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,14,16781,"{'yet': 675, 'owned': 10952, 'beaten': 4236, '...",86.0,10,651,2024-08-25T15:59:03,NaN,3925,0f0f0f,0f0f0f,"[{'platform': {'id': 18, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 33824, 'store': {'id': 7, 'name': 'Xbo...",NaN,"[{'id': 31, 'name': 'Singl

In [23]:
for store in all_stores:
    df_games[store] = df_games['store_names'].apply(lambda x: store in x)
df_games

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,...,itch.io,Nintendo Store,GOG,PlayStation Store,Google Play,App Store,Xbox Store,Steam,Epic Games,Xbox 360 Store
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,...,False,False,False,True,False,False,True,True,True,True
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,...,False,True,True,True,False,False,True,True,False,False
2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,...,False,False,False,True,False,False,True,True,False,True
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,...,False,False,False,True,False,False,False,True,False,True
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,...,False,False,False,True,True,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31075,960362,tiny-life-2,Tiny Life,2023-05-03,False,https://media.rawg.io/media/screenshots/b8d/b8...,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",2,...,False,False,False,False,False,False,False,True,False,False
31076,960061,dungeon-drafters-2,Dungeon Drafters,2023-04-26,False,https://media.rawg.io/media/screenshots/53c/53...,0.00,0,[],0,...,False,False,True,False,False,False,False,True,False,False
31077,959743,havendock,Havendock,None,False,https://media.rawg.io/media/screenshots/93d/93...,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",2,...,False,False,False,False,False,False,False,True,False,False
31078,946626,apewar,Apewar,2023-03-24,False,https://media.rawg.io/media/screenshots/2e2/2e...,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 1, 'percen...",2,...,False,False,False,False,False,False,False,True,False,False


In [121]:
df_games['rating_top'].value_counts()

rating_top
0    420
4    382
5    145
3    122
1     39
Name: count, dtype: int64

In [122]:
df_games['ratings'][0]

[{'id': 5, 'title': 'exceptional', 'count': 4114, 'percent': 59.1},
 {'id': 4, 'title': 'recommended', 'count': 2276, 'percent': 32.7},
 {'id': 3, 'title': 'meh', 'count': 442, 'percent': 6.35},
 {'id': 1, 'title': 'skip', 'count': 129, 'percent': 1.85}]

In [123]:
df_games = df_games.drop(columns='ratings')

In [124]:
df_games['added_by_status']

0       {'yet': 531, 'owned': 12152, 'beaten': 5971, 'toplay': 611, 'dropped': 1108, 'playing': 730}
1       {'yet': 1123, 'owned': 12006, 'beaten': 4848, 'toplay': 791, 'dropped': 946, 'playing': 873}
2        {'yet': 498, 'owned': 9519, 'beaten': 3755, 'toplay': 411, 'dropped': 1319, 'playing': 388}
3        {'yet': 452, 'owned': 10459, 'beaten': 771, 'toplay': 166, 'dropped': 1518, 'playing': 408}
4        {'yet': 619, 'owned': 8544, 'beaten': 2260, 'toplay': 422, 'dropped': 1321, 'playing': 315}
                                                    ...                                             
1103                                                                                              {}
1104                                                                                            None
1105                                                                                            None
1106                                                                                       

In [125]:
df_games.drop(columns='rating',inplace=True)

In [126]:
df_games['game_id']

0         3498
1         3328
2         5679
3           32
4         3070
         ...  
1103    795327
1104    887670
1105    979384
1106    968275
1107    917755
Name: game_id, Length: 1108, dtype: int64

In [226]:
df

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,reviews_text_count,added,added_by_status,metacritic,playtime,suggestions_count,updated,user_game,reviews_count,saturated_color,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,"{'yet': 532, 'owned': 12160, 'beaten': 5978, '...",92.0,74,434,2024-08-29T07:36:57,NaN,6964,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290375, 'store': {'id': 3, 'name': 'Pl...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902]
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,"{'yet': 1123, 'owned': 12013, 'beaten': 4855, ...",92.0,45,677,2024-08-28T20:12:31,NaN,6715,0f0f0f,0f0f0f,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]"
2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,"{'yet': 625, 'owned': 12214, 'beaten': 5519, '...",95.0,11,553,2024-08-29T07:23:05,NaN,5737,0f0f0f,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]"
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,"{'yet': 262, 'owned': 13146, 'beaten': 1048, '...",81.0,64,589,2024-08-20T10:48:54,NaN,3515,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498]
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 

In [366]:
df1 = df.drop(columns=['saturated_color', 'dominant_color', 'platforms', 'parent_platforms', 'genres', 'stores', 'clip', 'short_screenshots'])

In [228]:
df1.columns

Index(['id', 'slug', 'name', 'released', 'tba', 'background_image', 'rating',
       'rating_top', 'ratings', 'ratings_count', 'reviews_text_count', 'added',
       'added_by_status', 'metacritic', 'playtime', 'suggestions_count',
       'updated', 'user_game', 'reviews_count', 'tags', 'esrb_rating',
       'community_rating', 'platform_names', 'games_counts', 'store_names',
       'games_counts_stores', 'genre_names', 'games_counts_genres'],
      dtype='object')

In [316]:
df['esrb_rating'][0]

"{'id': 4, 'name': 'Mature', 'slug': 'mature'}"

In [272]:
df1.isna().sum()

id                         0
slug                       0
name                       0
released                1472
tba                        0
background_image         112
rating                     0
rating_top                 0
ratings                    0
ratings_count              0
reviews_text_count         0
added                      0
added_by_status            0
metacritic             24846
playtime                   0
suggestions_count          0
updated                    0
user_game              31080
reviews_count              0
tags                       0
esrb_rating            21839
community_rating       17903
platform_names             0
games_counts               0
store_names                0
games_counts_stores        0
genre_names                0
games_counts_genres        0
dtype: int64

In [367]:
df1 = df1.drop(columns=['background_image', 'user_game', 'community_rating'])

In [318]:
df1['metacritic'].mean()

73.0725056143728

In [241]:
df1['metacritic'].mode()

0    80.0
Name: metacritic, dtype: float64

In [368]:
df1['metacritic'] = df1['metacritic'].fillna(73)

In [321]:
df1['esrb_rating'][0]

"{'id': 4, 'name': 'Mature', 'slug': 'mature'}"

In [369]:
df2 = df1.dropna()

In [370]:
df2['esrb_rating'] = df2['esrb_rating'].apply(ast.literal_eval)

/var/folders/zc/6rc7nzhj13scy5qssxw2pjlh0000gn/T/ipykernel_1230/2526039268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['esrb_rating'] = df2['esrb_rating'].apply(ast.literal_eval)


In [303]:
df2['esrb_rating']

0            {'id': 4, 'name': 'Mature', 'slug': 'mature'}
1            {'id': 4, 'name': 'Mature', 'slug': 'mature'}
2        {'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...
3            {'id': 4, 'name': 'Mature', 'slug': 'mature'}
4            {'id': 4, 'name': 'Mature', 'slug': 'mature'}
                               ...                        
31058    {'id': 1, 'name': 'Everyone', 'slug': 'everyone'}
31059    {'id': 1, 'name': 'Everyone', 'slug': 'everyone'}
31067        {'id': 4, 'name': 'Mature', 'slug': 'mature'}
31071    {'id': 5, 'name': 'Adults Only', 'slug': 'adul...
31073    {'id': 1, 'name': 'Everyone', 'slug': 'everyone'}
Name: esrb_rating, Length: 9153, dtype: object

In [ ]:
def extract_platform_names(genres):
    if not isinstance(genres, list):
        return []
    return [genre['name'] for genre in genres]

def extract_games_count(genres):
    if not isinstance(genres, list):
        return []
    return [genre['games_count'] for genre in genres]

df['genre_names'] = df['genres'].apply(extract_platform_names)
df['games_counts_genres'] = df['genres'].apply(extract_games_count)

df[['name', 'genre_names', 'games_counts_genres']]

In [299]:
def extract_age_ratings(esrb_ratings):
    # Verifica si es una lista
    if not isinstance(esrb_ratings, list):
        return []
    # Extrae el nombre de cada rating de edad
    return [rating['name'] for rating in esrb_ratings]

# Aplica la función a la columna 'esrb_rating'
df1['age_rating'] = df1['esrb_rating'].apply(extract_age_ratings)

# Muestra el DataFrame con las columnas deseadas
df1[['name', 'age_rating']]


,name,age_rating
0,Grand Theft Auto V,[]
1,The Witcher 3: Wild Hunt,[]
2,Portal 2,[]
3,Counter-Strike: Global Offensive,[]
4,Tomb Raider (2013),[]
...,...,...
31075,Tiny Life,[]
31076,Dungeon Drafters,[]
31077,Havendock,[]
31078,Apewar,[]


In [327]:
df2 = df1.dropna()

In [385]:
df2

,id,slug,name,released,tba,rating,rating_top,ratings,ratings_count,reviews_text_count,added,added_by_status,metacritic,playtime,suggestions_count,updated,reviews_count,tags,esrb_rating,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,tags_names,tags_games_counts
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,"{'yet': 532, 'owned': 12160, 'beaten': 5978, '...",92.0,74,434,2024-08-29T07:36:57,6964,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902],"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216..."
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,"{'yet': 1123, 'owned': 12013, 'beaten': 4855, ...",92.0,45,677,2024-08-28T20:12:31,6715,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]","[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770..."
2,4200,portal-2,Portal 2,2011-04-18,False,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,"{'yet': 625, 'owned': 12214, 'beaten': 5519, '...",95.0,11,553,2024-08-29T07:23:05,5737,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]","[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75..."
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,"{'yet': 262, 'owned': 13146, 'beaten': 1048, '...",81.0,64,589,2024-08-20T10:48:54,3515,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498],"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327..."
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,14,16781,"{'yet': 675, 'owned': 10952, 'beaten': 4236, '...",86.0,10,651,2024-08-25T15:59:03,3925,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","[Action, Fighting]","[181902, 11756]","[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,1333,orbit,ORBIT,2011-12-06,False,0.00,0,[],0,0,18,"{'yet': 3, 'owned': 15}",73.0,1,248,2023-11-04T01:00:08,0,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 1, 'name': 'Everyone', 'slug': 'everyone'}","[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","[Action, Indie, Puzzle]","[181

In [387]:
df2['esrb_rating']

0            {'id': 4, 'name': 'Mature', 'slug': 'mature'}
1            {'id': 4, 'name': 'Mature', 'slug': 'mature'}
2        {'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...
3            {'id': 4, 'name': 'Mature', 'slug': 'mature'}
4            {'id': 4, 'name': 'Mature', 'slug': 'mature'}
                               ...                        
31058    {'id': 1, 'name': 'Everyone', 'slug': 'everyone'}
31059    {'id': 1, 'name': 'Everyone', 'slug': 'everyone'}
31067        {'id': 4, 'name': 'Mature', 'slug': 'mature'}
31071    {'id': 5, 'name': 'Adults Only', 'slug': 'adul...
31073    {'id': 1, 'name': 'Everyone', 'slug': 'everyone'}
Name: esrb_rating, Length: 9153, dtype: object

In [393]:
def extract_platform_names(esrb_ratings):
    if not isinstance(esrb_ratings, dict):
        print(esrb_ratings)
        return esrb_ratings.split(',')[1].split("'")[3]
    return esrb_ratings['name']

df2['age_rating'] = df2['esrb_rating'].apply(lambda x : extract_platform_names(x))

df2[['name', 'age_rating']]

/var/folders/zc/6rc7nzhj13scy5qssxw2pjlh0000gn/T/ipykernel_1230/3653424451.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['age_rating'] = df2['esrb_rating'].apply(lambda x : extract_platform_names(x))


,name,age_rating
0,Grand Theft Auto V,Mature
1,The Witcher 3: Wild Hunt,Mature
2,Portal 2,Everyone 10+
3,Counter-Strike: Global Offensive,Mature
4,Tomb Raider (2013),Mature
...,...,...
31058,ORBIT,Everyone
31059,Atari Flashback Classics Vol. 1,Everyone
31067,The Casting of Frank Stone,Mature
31071,Quake II: Enhanced Edition,Adults Only


In [394]:
df2

,id,slug,name,released,tba,rating,rating_top,ratings,ratings_count,reviews_text_count,added,added_by_status,metacritic,playtime,suggestions_count,updated,reviews_count,tags,esrb_rating,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,tags_names,tags_games_counts,age_rating
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,"{'yet': 532, 'owned': 12160, 'beaten': 5978, '...",92.0,74,434,2024-08-29T07:36:57,6964,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902],"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216...",Mature
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,"{'yet': 1123, 'owned': 12013, 'beaten': 4855, ...",92.0,45,677,2024-08-28T20:12:31,6715,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]","[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770...",Mature
2,4200,portal-2,Portal 2,2011-04-18,False,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,"{'yet': 625, 'owned': 12214, 'beaten': 5519, '...",95.0,11,553,2024-08-29T07:23:05,5737,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]","[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75...",Everyone 10+
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,"{'yet': 262, 'owned': 13146, 'beaten': 1048, '...",81.0,64,589,2024-08-20T10:48:54,3515,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498],"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327...",Mature
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,14,16781,"{'yet': 675, 'owned': 10952, 'beaten': 4236, '...",86.0,10,651,2024-08-25T15:59:03,3925,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","[Action, Fighting]","[181902, 11756]","[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23...",Mature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,1333,orbit,ORBIT,2011-12-06,False,0.00,0,[],0,0,18,"{'yet': 3, 'owned': 15}",73.0,1,248,2023-11-04T01:00:08,0,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 1, 'name': 'Everyone', 'slug': 'everyone'}","[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4

In [395]:
df2 = df2.drop(columns=['esrb_rating', 'added_by_status'])

In [396]:
df2

,id,slug,name,released,tba,rating,rating_top,ratings,ratings_count,reviews_text_count,added,metacritic,playtime,suggestions_count,updated,reviews_count,tags,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,tags_names,tags_games_counts,age_rating
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,92.0,74,434,2024-08-29T07:36:57,6964,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902],"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216...",Mature
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,92.0,45,677,2024-08-28T20:12:31,6715,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]","[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770...",Mature
2,4200,portal-2,Portal 2,2011-04-18,False,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,95.0,11,553,2024-08-29T07:23:05,5737,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]","[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75...",Everyone 10+
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,81.0,64,589,2024-08-20T10:48:54,3515,"[{'id': 40847, 'name': 'Steam Achievements', '...","[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498],"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327...",Mature
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,14,16781,86.0,10,651,2024-08-25T15:59:03,3925,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","[Action, Fighting]","[181902, 11756]","[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23...",Mature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,1333,orbit,ORBIT,2011-12-06,False,0.00,0,[],0,0,18,73.0,1,248,2023-11-04T01:00:08,0,"[{'id': 40847, 'name': 'Steam Achievements', '...","[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","[Action, Indie, Puzzle]","[181902, 68528, 97296]","[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 32327, 196856, 11975, 68...",Everyone
31059,1052,atari-flashback-classics-vol-1,Atari Flashback Classics Vol. 1,2016-10-13,False,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",3,0,18,73.0,0,103,2022-09-09T08:34:23,3,"[{'id': 7, 'name': 'Multiplayer', 'slug': 'mul...","[Xbox One, PlayStation 4]","[5638, 6825]","[PlayStation Store, Xbox Store]","[7961, 4865]",[Arcade],[22658],"[Multiplayer, online, console, history, bundle...","[38545, 6557, 1390, 2374,

In [397]:
df2.isna().sum()

id                     0
slug                   0
name                   0
released               0
tba                    0
rating                 0
rating_top             0
ratings                0
ratings_count          0
reviews_text_count     0
added                  0
metacritic             0
playtime               0
suggestions_count      0
updated                0
reviews_count          0
tags                   0
platform_names         0
games_counts           0
store_names            0
games_counts_stores    0
genre_names            0
games_counts_genres    0
tags_names             0
tags_games_counts      0
age_rating             0
dtype: int64

In [341]:
df2['tags'][0]

"[{'id': 31, 'name': 'Singleplayer', 'slug': 'singleplayer', 'language': 'eng', 'games_count': 226836, 'image_background': 'https://media.rawg.io/media/games/562/562553814dd54e001a541e4ee83a591c.jpg'}, {'id': 40847, 'name': 'Steam Achievements', 'slug': 'steam-achievements', 'language': 'eng', 'games_count': 39362, 'image_background': 'https://media.rawg.io/media/games/b8c/b8c243eaa0fbac8115e0cdccac3f91dc.jpg'}, {'id': 7, 'name': 'Multiplayer', 'slug': 'multiplayer', 'language': 'eng', 'games_count': 38545, 'image_background': 'https://media.rawg.io/media/games/713/713269608dc8f2f40f5a670a14b2de94.jpg'}, {'id': 40836, 'name': 'Full controller support', 'slug': 'full-controller-support', 'language': 'eng', 'games_count': 18689, 'image_background': 'https://media.rawg.io/media/games/942/9424d6bb763dc38d9378b488603c87fa.jpg'}, {'id': 13, 'name': 'Atmospheric', 'slug': 'atmospheric', 'language': 'eng', 'games_count': 34016, 'image_background': 'https://media.rawg.io/media/games/fc1/fc1307a

In [398]:
df2['tags'] = df2['tags'].apply(ast.literal_eval)

ValueError: malformed node or string: [{'id': 31, 'name': 'Singleplayer', 'slug': 'singleplayer', 'language': 'eng', 'games_count': 226836, 'image_background': 'https://media.rawg.io/media/games/562/562553814dd54e001a541e4ee83a591c.jpg'}, {'id': 40847, 'name': 'Steam Achievements', 'slug': 'steam-achievements', 'language': 'eng', 'games_count': 39362, 'image_background': 'https://media.rawg.io/media/games/b8c/b8c243eaa0fbac8115e0cdccac3f91dc.jpg'}, {'id': 7, 'name': 'Multiplayer', 'slug': 'multiplayer', 'language': 'eng', 'games_count': 38545, 'image_background': 'https://media.rawg.io/media/games/713/713269608dc8f2f40f5a670a14b2de94.jpg'}, {'id': 40836, 'name': 'Full controller support', 'slug': 'full-controller-support', 'language': 'eng', 'games_count': 18689, 'image_background': 'https://media.rawg.io/media/games/942/9424d6bb763dc38d9378b488603c87fa.jpg'}, {'id': 13, 'name': 'Atmospheric', 'slug': 'atmospheric', 'language': 'eng', 'games_count': 34016, 'image_background': 'https://

In [399]:
def extract_platform_names(tags):
    if not isinstance(tags, list):
        return []
    return [tag['name'] for tag in tags]

def extract_tags_count(tags):
    if not isinstance(tags, list):
        return []
    return [tag['games_count'] for tag in tags]

df2['tags_names'] = df2['tags'].apply(extract_platform_names)
df2['tags_games_counts'] = df2['tags'].apply(extract_tags_count)

df2[['name', 'tags_names', 'tags_games_counts']]

,name,tags_names,tags_games_counts
0,Grand Theft Auto V,"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216..."
1,The Witcher 3: Wild Hunt,"[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770..."
2,Portal 2,"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75..."
3,Counter-Strike: Global Offensive,"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327..."
4,Tomb Raider (2013),"[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23..."
...,...,...,...
31058,ORBIT,"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 32327, 196856, 11975, 68..."
31059,Atari Flashback Classics Vol. 1,"[Multiplayer, online, console, history, bundle...","[38545, 6557, 1390, 2374, 478, 1157]"
31067,The Casting of Frank Stone,[],[]
31071,Quake II: Enhanced Edition,"[Singleplayer, First-Person, Sci-fi, Horror, F...","[226945, 32343, 19510, 44922, 13462, 1794, 418..."


In [400]:
df3 = df2.drop(columns=['id', 'tags'])

In [402]:
df3

,slug,name,released,tba,rating,rating_top,ratings,ratings_count,reviews_text_count,added,metacritic,playtime,suggestions_count,updated,reviews_count,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,tags_names,tags_games_counts,age_rating
0,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,92.0,74,434,2024-08-29T07:36:57,6964,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902],"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216...",Mature
1,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,92.0,45,677,2024-08-28T20:12:31,6715,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]","[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770...",Mature
2,portal-2,Portal 2,2011-04-18,False,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,95.0,11,553,2024-08-29T07:23:05,5737,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]","[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75...",Everyone 10+
3,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,81.0,64,589,2024-08-20T10:48:54,3515,"[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498],"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327...",Mature
4,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,14,16781,86.0,10,651,2024-08-25T15:59:03,3925,"[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","[Action, Fighting]","[181902, 11756]","[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23...",Mature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,orbit,ORBIT,2011-12-06,False,0.00,0,[],0,0,18,73.0,1,248,2023-11-04T01:00:08,0,"[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","[Action, Indie, Puzzle]","[181902, 68528, 97296]","[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 32327, 196856, 11975, 68...",Everyone
31059,atari-flashback-classics-vol-1,Atari Flashback Classics Vol. 1,2016-10-13,False,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",3,0,18,73.0,0,103,2022-09-09T08:34:23,3,"[Xbox One, PlayStation 4]","[5638, 6825]","[PlayStation Store, Xbox Store]","[7961, 4865]",[Arcade],[22658],"[Multiplayer, online, console, history, bundle...","[38545, 6557, 1390, 2374, 478, 1157]",Everyone
31067,the-casting-of-frank-stone,The Casting of Frank Stone,2024-09-03,False,0.00,0,[],0,0,17,73.0,0,297,2024-07-17T07:08:32,0,"[PlayStation 5, PC, Xbox Series S/X]","[1107, 535134, 973]","[Xbox Store, PlayStation Store, Epic Games, St...","[4867, 7962, 1345, 98196]",[Adventure],[141620],[],[],Mature
31071,quake-ii-enhanced-edition,Quake II: Enhanced Edition,2023-08-10,False,4.71,5,"[{'id':

In [353]:
df3

,slug,name,released,tba,rating,rating_top,ratings,ratings_count,reviews_text_count,added,metacritic,playtime,suggestions_count,updated,reviews_count,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,age_rating,tags_names,tags_games_counts
0,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,92.0,74,434,2024-08-29T07:36:57,6964,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902],Mature,"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216..."
1,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,92.0,45,677,2024-08-28T20:12:31,6715,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]",Mature,"[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770..."
2,portal-2,Portal 2,2011-04-18,False,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,95.0,11,553,2024-08-29T07:23:05,5737,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]",Everyone 10+,"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75..."
3,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,81.0,64,589,2024-08-20T10:48:54,3515,"[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498],Mature,"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327..."
4,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,14,16781,86.0,10,651,2024-08-25T15:59:03,3925,"[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","[Action, Fighting]","[181902, 11756]",Mature,"[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,orbit,ORBIT,2011-12-06,False,0.00,0,[],0,0,18,73.0,1,248,2023-11-04T01:00:08,0,"[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","[Action, Indie, Puzzle]","[181902, 68528, 97296]",Everyone,"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 32327, 196856, 11975, 68..."
31059,atari-flashback-classics-vol-1,Atari Flashback Classics Vol. 1,2016-10-13,False,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",3,0,18,73.0,0,103,2022-09-09T08:34:23,3,"[Xbox One, PlayStation 4]","[5638, 6825]","[PlayStation Store, Xbox Store]","[7961, 4865]",[Arcade],[22658],Everyone,"[Multiplayer, online, console, history, bundle...","[38545, 6557, 1390, 2374, 478, 1157]"
31067,the-casting-of-frank-stone,The Casting of Frank Stone,2024-09-03,False,0.00,0,[],0,0,17,73.0,0,297,2024-07-17T07:08:32,0,"[PlayStation 5, PC, Xbox Series S/X]","[1107, 535134, 973]","[Xbox Store, PlayStation Store, Epic Games, St...","[4867, 7962, 1345, 98196]",[Adventure],[141620],Mature,[],[]
31071,quake-ii-enhanced-edition,Quake II: Enhanced Edition,2023-08-10,False,4.71,5,"[{'id':

In [24]:
status_df = pd.json_normalize(df['added_by_status'])
df_final = pd.concat([df_games.drop('added_by_status', axis=1), status_df], axis=1)

df_final.columns


Index(['id', 'slug', 'name', 'released', 'tba', 'background_image', 'rating',
       'rating_top', 'ratings', 'ratings_count', 'reviews_text_count', 'added',
       'metacritic', 'playtime', 'suggestions_count', 'updated', 'user_game',
       'reviews_count', 'saturated_color', 'dominant_color',
       'parent_platforms', 'genres', 'stores', 'clip', 'tags', 'esrb_rating',
       'short_screenshots', 'community_rating', 'genre_names', 'store_names',
       'platform_names', 'SEGA 32X', 'Nintendo 64', 'Atari 8-bit', 'PC',
       'SEGA Master System', 'Dreamcast', 'Atari 5200', 'PlayStation 2',
       'Xbox One', 'PlayStation', 'macOS', 'Wii', 'PSP', 'Atari Lynx',
       'Game Boy', 'Wii U', 'Nintendo 3DS', 'Commodore / Amiga',
       'Game Boy Color', 'Game Gear', 'Jaguar', 'Atari 7800', 'NES',
       'Nintendo Switch', 'Atari 2600', 'Linux', 'Xbox Series S/X', '3DO',
       'Nintendo DS', 'Xbox 360', 'Xbox', 'Nintendo DSi', 'Web', 'SNES',
       'GameCube', 'SEGA Saturn', 'Atari Flashba

In [128]:
df_final['game_id']

0         3498
1         3328
2         5679
3           32
4         3070
         ...  
1103    795327
1104    887670
1105    979384
1106    968275
1107    917755
Name: game_id, Length: 1108, dtype: int64

In [129]:
df_games.columns

Index(['name', 'playtime', 'released', 'tba', 'rating_top', 'ratings_count',
       'reviews_text_count', 'added', 'added_by_status', 'metacritic',
       'suggestions_count', 'updated', 'score', 'clip', 'tags', 'esrb_rating',
       'user_game', 'reviews_count', 'saturated_color', 'dominant_color',
       'parent_platforms', 'community_rating', 'platform_names', 'game_id',
       'store_names', 'genre_names', 'SNES', 'Wii U', 'Wii', 'PSP',
       'PlayStation 3', 'Nintendo Switch', 'Xbox Series S/X', 'Xbox One',
       'Xbox 360', 'PlayStation 2', 'Game Boy Advance', 'Nintendo 3DS',
       'SEGA Saturn', 'Nintendo DSi', 'PS Vita', 'Genesis', 'Android',
       'Commodore / Amiga', 'Classic Macintosh', 'Web', 'Nintendo 64',
       'Nintendo DS', 'Game Gear', 'macOS', 'Game Boy', 'PlayStation 5',
       'Xbox', 'Dreamcast', 'iOS', 'PC', 'PlayStation', 'GameCube',
       'PlayStation 4', 'Linux', 'SEGA Master System', 'Steam', 'App Store',
       'Xbox 360 Store', 'Xbox Store', 'Nintendo 

In [25]:
columns_with_lists = [col for col in df_final.columns if df_final[col].apply(lambda x: isinstance(x, dict)).any()]
columns_with_lists

['esrb_rating']

In [27]:
df_games

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,...,itch.io,Nintendo Store,GOG,PlayStation Store,Google Play,App Store,Xbox Store,Steam,Epic Games,Xbox 360 Store
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,...,False,False,False,True,False,False,True,True,True,True
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,...,False,True,True,True,False,False,True,True,False,False
2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,...,False,False,False,True,False,False,True,True,False,True
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,...,False,False,False,True,False,False,False,True,False,True
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,...,False,False,False,True,True,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31075,960362,tiny-life-2,Tiny Life,2023-05-03,False,https://media.rawg.io/media/screenshots/b8d/b8...,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",2,...,False,False,False,False,False,False,False,True,False,False
31076,960061,dungeon-drafters-2,Dungeon Drafters,2023-04-26,False,https://media.rawg.io/media/screenshots/53c/53...,0.00,0,[],0,...,False,False,True,False,False,False,False,True,False,False
31077,959743,havendock,Havendock,None,False,https://media.rawg.io/media/screenshots/93d/93...,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",2,...,False,False,False,False,False,False,False,True,False,False
31078,946626,apewar,Apewar,2023-03-24,False,https://media.rawg.io/media/screenshots/2e2/2e...,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 1, 'percen...",2,...,False,False,False,False,False,False,False,True,False,False


In [131]:
df_final['esrb_rating'].value_counts()

esrb_rating
{'id': 4, 'name': 'Mature', 'slug': 'mature', 'name_en': 'Mature', 'name_ru': 'С 17 лет'}                                       200
{'id': 3, 'name': 'Teen', 'slug': 'teen', 'name_en': 'Teen', 'name_ru': 'С 13 лет'}                                             165
{'id': 2, 'name': 'Everyone 10+', 'slug': 'everyone-10-plus', 'name_en': 'Everyone 10+', 'name_ru': 'С 10 лет'}                  70
{'id': 1, 'name': 'Everyone', 'slug': 'everyone', 'name_en': 'Everyone', 'name_ru': 'Для всех'}                                  66
{'id': 6, 'name': 'Rating Pending', 'slug': 'rating-pending', 'name_en': 'Rating Pending', 'name_ru': 'Рейтинг обсуждается'}     30
{'id': 5, 'name': 'Adults Only', 'slug': 'adults-only', 'name_en': 'Adults Only', 'name_ru': 'Только для взрослых'}              25
Name: count, dtype: int64

In [134]:
df

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,reviews_text_count,added,added_by_status,metacritic,playtime,suggestions_count,updated,user_game,reviews_count,saturated_color,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating,platform_names,store_names,genre_names
0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,"{'yet': 532, 'owned': 12160, 'beaten': 5978, '...",92.0,74,434,2024-08-29T07:36:57,NaN,6964,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290375, 'store': {'id': 3, 'name': 'Pl...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[PlayStation Store, Epic Games, Steam, Xbox 36...",[Action]
1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,"{'yet': 1123, 'owned': 12013, 'beaten': 4855, ...",92.0,45,677,2024-08-28T20:12:31,NaN,6715,0f0f0f,0f0f0f,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[Action, RPG]"
2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,"{'yet': 625, 'owned': 12214, 'beaten': 5519, '...",95.0,11,553,2024-08-29T07:23:05,NaN,5737,0f0f0f,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[Shooter, Puzzle]"
3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,"{'yet': 262, 'owned': 13146, 'beaten': 1048, '...",81.0,64,589,2024-08-20T10:48:54,NaN,3515,0f0f0f,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,"[PC, Linux, Xbox 360, PlayStation 3]","[PlayStation Store, Steam, Xbox 360 Store]",[Shooter]
4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,14,16781,"{'yet': 675, 'owned': 10952, 'beaten': 4236, '...",86.0,10,651,2024-08-25T15:59:03,NaN,3925,0f0f0f,0f0f0f,"[{'platform': {'id': 18, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 33824, 'store': {'id': 7, 'name': 'Xbo...",NaN,"[{'id': 31, 'name': 'Singl

In [133]:
df['esrb_rating'] = df['esrb_rating'].apply(ast.literal_eval)

ValueError: malformed node or string: nan

In [132]:
esrb_rating = pd.json_normalize(df['esrb_rating'])
esrb_rating
esrb_rating_name=esrb_rating.drop(columns=['slug', 'name_en', 'name_ru', 'id'])
esrb_rating_name['age rating'] = esrb_rating_name['name']

KeyError: "['slug', 'name_en', 'name_ru', 'id'] not found in axis"

In [133]:
df_final1 = pd.concat([df_final.drop('esrb_rating', axis=1), esrb_rating_name], axis=1)

df_final1.columns

Index(['name', 'playtime', 'released', 'tba', 'rating_top', 'ratings_count',
       'reviews_text_count', 'added', 'metacritic', 'suggestions_count',
       'updated', 'score', 'clip', 'tags', 'user_game', 'reviews_count',
       'saturated_color', 'dominant_color', 'parent_platforms',
       'community_rating', 'platform_names', 'game_id', 'store_names',
       'genre_names', 'SNES', 'Wii U', 'Wii', 'PSP', 'PlayStation 3',
       'Nintendo Switch', 'Xbox Series S/X', 'Xbox One', 'Xbox 360',
       'PlayStation 2', 'Game Boy Advance', 'Nintendo 3DS', 'SEGA Saturn',
       'Nintendo DSi', 'PS Vita', 'Genesis', 'Android', 'Commodore / Amiga',
       'Classic Macintosh', 'Web', 'Nintendo 64', 'Nintendo DS', 'Game Gear',
       'macOS', 'Game Boy', 'PlayStation 5', 'Xbox', 'Dreamcast', 'iOS', 'PC',
       'PlayStation', 'GameCube', 'PlayStation 4', 'Linux',
       'SEGA Master System', 'Steam', 'App Store', 'Xbox 360 Store',
       'Xbox Store', 'Nintendo Store', 'Google Play', 'PlayStatio

In [134]:
df_final1['game_id']

0         3498
1         3328
2         5679
3           32
4         3070
         ...  
1103    795327
1104    887670
1105    979384
1106    968275
1107    917755
Name: game_id, Length: 1108, dtype: int64

In [135]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [136]:
df_final1 = df_final1.drop(columns=['platform_names', 'store_names'])


In [137]:
df_final1.drop(columns='parent_platforms', inplace=True)

In [138]:
df_final1['genre_names']

0                         [Action]
1                    [Action, RPG]
2                    [Action, RPG]
3                [Shooter, Action]
4                    [Action, RPG]
                   ...            
1103           [Adventure, Action]
1104                         [RPG]
1105    [Casual, Indie, Adventure]
1106                      [Arcade]
1107                 [Action, RPG]
Name: genre_names, Length: 1108, dtype: object

In [139]:
df_final1.shape

(1108, 74)

In [191]:

df3['genre_names'] = df3['genre_names'].astype(str).replace('nan', '')

In [354]:
df3.dtypes

slug                    object
name                    object
released                object
tba                       bool
rating                 float64
rating_top               int64
ratings                 object
ratings_count            int64
reviews_text_count       int64
added                    int64
metacritic             float64
playtime                 int64
suggestions_count        int64
updated                 object
reviews_count            int64
platform_names          object
games_counts            object
store_names             object
games_counts_stores     object
genre_names             object
games_counts_genres     object
age_rating              object
tags_names              object
tags_games_counts       object
dtype: object

In [383]:
df3

,slug,name,released,tba,rating,rating_top,ratings,ratings_count,reviews_text_count,added,added_by_status,metacritic,playtime,suggestions_count,updated,reviews_count,esrb_rating,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,tags_names,tags_games_counts
0,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",6858,62,21120,"{'yet': 532, 'owned': 12160, 'beaten': 5978, '...",92.0,74,434,2024-08-29T07:36:57,6964,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902],"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216..."
1,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,"[{'id': 5, 'title': 'exceptional', 'count': 51...",6606,75,20603,"{'yet': 1123, 'owned': 12013, 'beaten': 4855, ...",92.0,45,677,2024-08-28T20:12:31,6715,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]","[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770..."
2,portal-2,Portal 2,2011-04-18,False,4.60,5,"[{'id': 5, 'title': 'exceptional', 'count': 40...",5685,35,19487,"{'yet': 625, 'owned': 12214, 'beaten': 5519, '...",95.0,11,553,2024-08-29T07:23:05,5737,"{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]","[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75..."
3,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3479,27,17173,"{'yet': 262, 'owned': 13146, 'beaten': 1048, '...",81.0,64,589,2024-08-20T10:48:54,3515,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498],"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327..."
4,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 23...",3896,14,16781,"{'yet': 675, 'owned': 10952, 'beaten': 4236, '...",86.0,10,651,2024-08-25T15:59:03,3925,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","[Action, Fighting]","[181902, 11756]","[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,orbit,ORBIT,2011-12-06,False,0.00,0,[],0,0,18,"{'yet': 3, 'owned': 15}",73.0,1,248,2023-11-04T01:00:08,0,"{'id': 1, 'name': 'Everyone', 'slug': 'everyone'}","[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","[Action, Indie, Puzzle]","[181902, 68528, 97296]","[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 32327, 196856, 11975, 68..."
31059,atari-flashback-classics-vol-1,Atari Flashback Classics Vol. 1,2016-10-13,False,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",3,0,18,"{'owned': 12, 'beaten': 2, 'toplay': 1, 'dropp...",73.0,0,103,2022-09-09T08:34:23

In [403]:
df3.drop(columns='ratings',inplace=True)

In [404]:
df3

,slug,name,released,tba,rating,rating_top,ratings_count,reviews_text_count,added,metacritic,playtime,suggestions_count,updated,reviews_count,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,tags_names,tags_games_counts,age_rating
0,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,6858,62,21120,92.0,74,434,2024-08-29T07:36:57,6964,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",[Action],[181902],"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216...",Mature
1,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,6606,75,20603,92.0,45,677,2024-08-28T20:12:31,6715,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","[Action, RPG]","[181902, 56940]","[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770...",Mature
2,portal-2,Portal 2,2011-04-18,False,4.60,5,5685,35,19487,95.0,11,553,2024-08-29T07:23:05,5737,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","[Shooter, Puzzle]","[59498, 97296]","[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75...",Everyone 10+
3,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,3479,27,17173,81.0,64,589,2024-08-20T10:48:54,3515,"[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",[Shooter],[59498],"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327...",Mature
4,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,3896,14,16781,86.0,10,651,2024-08-25T15:59:03,3925,"[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","[Action, Fighting]","[181902, 11756]","[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23...",Mature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,orbit,ORBIT,2011-12-06,False,0.00,0,0,0,18,73.0,1,248,2023-11-04T01:00:08,0,"[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","[Action, Indie, Puzzle]","[181902, 68528, 97296]","[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 32327, 196856, 11975, 68...",Everyone
31059,atari-flashback-classics-vol-1,Atari Flashback Classics Vol. 1,2016-10-13,False,0.00,0,3,0,18,73.0,0,103,2022-09-09T08:34:23,3,"[Xbox One, PlayStation 4]","[5638, 6825]","[PlayStation Store, Xbox Store]","[7961, 4865]",[Arcade],[22658],"[Multiplayer, online, console, history, bundle...","[38545, 6557, 1390, 2374, 478, 1157]",Everyone
31067,the-casting-of-frank-stone,The Casting of Frank Stone,2024-09-03,False,0.00,0,0,0,17,73.0,0,297,2024-07-17T07:08:32,0,"[PlayStation 5, PC, Xbox Series S/X]","[1107, 535134, 973]","[Xbox Store, PlayStation Store, Epic Games, St...","[4867, 7962, 1345, 98196]",[Adventure],[141620],[],[],Mature
31071,quake-ii-enhanced-edition,Quake II: Enhanced Edition,2023-08-10,False,4.71,5,7,0,17,73.0,0,411,2024-07-12T08:19:16,7,"[Nintendo Switch, Xbox Series S/X, Xbox One, P...","[5504, 973, 5638, 6825, 1107, 535134]","[GOG, Epic Games, Steam]","[6290, 1345, 98196]","[Action, Shooter]","[181941, 59498]","[Singleplayer, First-Person, Sci-fi, Horror, F...","[226945, 32343, 19510, 44922, 13462, 1794, 418...",Adults Only


In [375]:
df3 = df3.drop(columns='added_by_status')

In [405]:
df3['genre_names'] = df3['genre_names'].astype(str)
df3['cleaned'] = df3['genre_names'].str.strip('[]')
split_data = df3['cleaned'].str.split(',', n=1, expand=True)
df3['genre'] = split_data[0].str.strip()
df3['subgenres'] = split_data[1].str.strip() if split_data.shape[1] > 1 else ''
df3['subgenres'] = df3['subgenres'].apply(lambda x: [sg.strip().strip("'") for sg in x.split(',')] if x else [])
for i in range(10):
    df3[f'subgenre_{i+1}'] = df3['subgenres'].apply(lambda x: x[i] if i < len(x) else None)
df3 = df3.drop(columns=['cleaned', 'subgenres'])

df3


,slug,name,released,tba,rating,rating_top,ratings_count,reviews_text_count,added,metacritic,playtime,suggestions_count,updated,reviews_count,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,tags_names,tags_games_counts,age_rating,genre,subgenre_1,subgenre_2,subgenre_3,subgenre_4,subgenre_5,subgenre_6,subgenre_7,subgenre_8,subgenre_9,subgenre_10
0,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,6858,62,21120,92.0,74,434,2024-08-29T07:36:57,6964,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",['Action'],[181902],"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216...",Mature,'Action',None,None,None,None,None,None,None,None,None,None
1,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,6606,75,20603,92.0,45,677,2024-08-28T20:12:31,6715,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","['Action', 'RPG']","[181902, 56940]","[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770...",Mature,'Action',RPG,None,None,None,None,None,None,None,None,None
2,portal-2,Portal 2,2011-04-18,False,4.60,5,5685,35,19487,95.0,11,553,2024-08-29T07:23:05,5737,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","['Shooter', 'Puzzle']","[59498, 97296]","[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75...",Everyone 10+,'Shooter',Puzzle,None,None,None,None,None,None,None,None,None
3,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,3479,27,17173,81.0,64,589,2024-08-20T10:48:54,3515,"[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",['Shooter'],[59498],"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327...",Mature,'Shooter',None,None,None,None,None,None,None,None,None,None
4,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,3896,14,16781,86.0,10,651,2024-08-25T15:59:03,3925,"[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","['Action', 'Fighting']","[181902, 11756]","[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23...",Mature,'Action',Fighting,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,orbit,ORBIT,2011-12-06,False,0.00,0,0,0,18,73.0,1,248,2023-11-04T01:00:08,0,"[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","['Action', 'Indie', 'Puzzle']","[181902, 68528, 97296]","[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 32327, 196856, 11975, 68...",Everyone,'Action',Indie,Puzzle,None,None,None,None,None,None,None,None
31059,atari-flashback-classics-vol-1,Atari Flashback Classics Vol. 1,2016-10-13,False,0.00,0,3,0,18,73.0,0,103,2022-09-09T08:34:23,3,"[Xbox One, PlayStation 4]","[5638, 6825]","[PlayStation Store, Xbox Store]","[7961, 4865]",['Arcade'],[22658],"[Multiplayer, online, console, history, bundle...","[38545, 6557, 1390, 2374, 478, 1157]",Everyone,'Arcade',None,None,None,None,None,None,None,None,None,None
31067,the-casting-of-frank-stone,The Casting of Frank Stone,2024-09-03,False,0.00,0,0,0,17,73.0,0,297,2024-07-17

In [206]:
df3['subgenre_10'].value_counts()

subgenre_10
Casual    1
Name: count, dtype: int64

In [207]:
df3['subgenre_10'].isna().sum()

31079

In [379]:
df3

,slug,name,released,tba,rating,rating_top,ratings_count,reviews_text_count,added,metacritic,playtime,suggestions_count,updated,reviews_count,esrb_rating,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,genre,subgenre_1,subgenre_2,subgenre_3,subgenre_4,subgenre_5,subgenre_6,subgenre_7,subgenre_8,subgenre_9,subgenre_10
0,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,6858,62,21120,92.0,74,434,2024-08-29T07:36:57,6964,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",['Action'],[181902],'Action',None,None,None,None,None,None,None,None,None,None
1,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,6606,75,20603,92.0,45,677,2024-08-28T20:12:31,6715,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","['Action', 'RPG']","[181902, 56940]",'Action',RPG,None,None,None,None,None,None,None,None,None
2,portal-2,Portal 2,2011-04-18,False,4.60,5,5685,35,19487,95.0,11,553,2024-08-29T07:23:05,5737,"{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","['Shooter', 'Puzzle']","[59498, 97296]",'Shooter',Puzzle,None,None,None,None,None,None,None,None,None
3,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,3479,27,17173,81.0,64,589,2024-08-20T10:48:54,3515,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",['Shooter'],[59498],'Shooter',None,None,None,None,None,None,None,None,None,None
4,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,3896,14,16781,86.0,10,651,2024-08-25T15:59:03,3925,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","['Action', 'Fighting']","[181902, 11756]",'Action',Fighting,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,orbit,ORBIT,2011-12-06,False,0.00,0,0,0,18,73.0,1,248,2023-11-04T01:00:08,0,"{'id': 1, 'name': 'Everyone', 'slug': 'everyone'}","[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","['Action', 'Indie', 'Puzzle']","[181902, 68528, 97296]",'Action',Indie,Puzzle,None,None,None,None,None,None,None,None
31059,atari-flashback-classics-vol-1,Atari Flashback Classics Vol. 1,2016-10-13,False,0.00,0,3,0,18,73.0,0,103,2022-09-09T08:34:23,3,"{'id': 1, 'name': 'Everyone', 'slug': 'everyone'}","[Xbox One, PlayStation 4]","[5638, 6825]","[PlayStation Store, Xbox Store]","[7961, 4865]",['Arcade'],[22658],'Arcade',None,None,None,None,None,None,None,None,None,None
31067,the-casting-of-frank-stone,The Casting of Frank Stone,2024-09-03,False,0.00,0,0,0,17,73.0,0,297,2024-07-17T07:08:32,0,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[PlayStation 5, PC, Xbox Series S/X]","[1107, 535134, 973]","[Xbox Store, PlayStation Store, Epic Games, St...","[4867, 7962, 1345, 98196]",['Adventure'],[141620],'Adventure',None,None,None,None,None,None,None,None,None,None
31071,quake-ii-enhanced-edition,Quake II: Enhanced Edition,2023-08-10,False,4.71,5,7,0,17,73.0,0,411,2024-07-12T08:19:16,7,"{'id': 5, 'name': 'Adults Only', 'slug': 'adul...","

In [406]:
df3.to_csv('df_idgb_info_juegos.csv',index=False)

In [378]:
df3.columns

Index(['slug', 'name', 'released', 'tba', 'rating', 'rating_top',
       'ratings_count', 'reviews_text_count', 'added', 'metacritic',
       'playtime', 'suggestions_count', 'updated', 'reviews_count',
       'esrb_rating', 'platform_names', 'games_counts', 'store_names',
       'games_counts_stores', 'genre_names', 'games_counts_genres', 'genre',
       'subgenre_1', 'subgenre_2', 'subgenre_3', 'subgenre_4', 'subgenre_5',
       'subgenre_6', 'subgenre_7', 'subgenre_8', 'subgenre_9', 'subgenre_10'],
      dtype='object')

In [145]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [146]:
df_clean1 = df

In [147]:
df_clean1['game_id']

0         3498
1         3328
2         5679
3           32
4         3070
         ...  
1103    795327
1104    887670
1105    979384
1106    968275
1107    917755
Name: game_id, Length: 1108, dtype: int64

In [148]:
df_clean1.shape

(1108, 66)

In [149]:
df_clean1

name    released  \
0                                          Grand Theft Auto V  2013-09-17   
1                                    The Witcher 3: Wild Hunt  2015-05-18   
2                                 The Elder Scrolls V: Skyrim  2011-11-11   
3                                                   Destiny 2  2017-09-06   
4                                                   Fallout 4  2015-11-09   
...                                                       ...         ...   
1103                      Dead by Daylight - Spark of Madness  2017-05-11   
1104  Pre-purchase Romancing SaGa - Minstrel Song- Remastered  2022-12-01   
1105                                 The Courier [PM Studios]        None   
1106                                             Neo Breakout  2022-11-11   
1107                                               Steel Seed        None   

        tba  rating_top  ratings_count  added  metacritic  suggestions_count  \
0     False           5           6855  21103        92.0                434   
1     False           5           6599  20587        92.0                677   
2     False           5           4632  15890        94.0                602   
3     False           4           2594  13774        82.0               1230   
4     False           4           3273  13481        84.0                460   
...     ...         ...            ...    ...         ...                ...   
1103  False           0              0      0         NaN                384   
1104  False           0              0      0         NaN                162   
1105   True           0              0      0         NaN                  0   
1106  False           0              0      0         NaN                  0   
1107   True           0              1      0         NaN                350   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [150]:
cols = list(df_clean1.columns)
cols[0]='game_name'


In [151]:
df_clean1.columns=cols

In [152]:
df_clean1 = df_clean1.drop(columns= ['name'])

In [153]:
df_clean1['genre'] = df_clean1['genre'].str.replace('"', '', regex=False).str.replace("'", '', regex=False)
df_clean1['subgenre'] = df_clean1['subgenre'].str.replace('"', '', regex=False).str.replace("'", '', regex=False)

In [154]:
df_clean1['genre'] = df_clean1['genre'].str.replace('[', '', regex=False).str.replace("]", '', regex=False)
df_clean1['subgenre'] = df_clean1['subgenre'].str.replace('[', '', regex=False).str.replace("]", '', regex=False)

In [155]:
df_clean1

game_name    released  \
0                                          Grand Theft Auto V  2013-09-17   
1                                    The Witcher 3: Wild Hunt  2015-05-18   
2                                 The Elder Scrolls V: Skyrim  2011-11-11   
3                                                   Destiny 2  2017-09-06   
4                                                   Fallout 4  2015-11-09   
...                                                       ...         ...   
1103                      Dead by Daylight - Spark of Madness  2017-05-11   
1104  Pre-purchase Romancing SaGa - Minstrel Song- Remastered  2022-12-01   
1105                                 The Courier [PM Studios]        None   
1106                                             Neo Breakout  2022-11-11   
1107                                               Steel Seed        None   

        tba  rating_top  ratings_count  added  metacritic  suggestions_count  \
0     False           5           6855  21103        92.0                434   
1     False           5           6599  20587        92.0                677   
2     False           5           4632  15890        94.0                602   
3     False           4           2594  13774        82.0               1230   
4     False           4           3273  13481        84.0                460   
...     ...         ...            ...    ...         ...                ...   
1103  False           0              0      0         NaN                384   
1104  False           0              0      0         NaN                162   
1105   True           0              0      0         NaN                  0   
1106  False           0              0      0         NaN                  0   
1107   True           0              1      0         NaN                350   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [171]:
df_clean1['tags']

[{'id': 31,
  'name': 'Singleplayer',
  'slug': 'singleplayer',
  'language': 'eng',
  'games_count': 226578,
  'image_background': 'https://media.rawg.io/media/games/b45/b45575f34285f2c4479c9a5f719d972e.jpg'},
 {'id': 42396,
  'name': 'Для одного игрока',
  'slug': 'dlia-odnogo-igroka',
  'language': 'rus',
  'games_count': 51026,
  'image_background': 'https://media.rawg.io/media/games/960/960b601d9541cec776c5fa42a00bf6c4.jpg'},
 {'id': 42417,
  'name': 'Экшен',
  'slug': 'ekshen',
  'language': 'rus',
  'games_count': 41030,
  'image_background': 'https://media.rawg.io/media/games/310/3106b0e012271c5ffb16497b070be739.jpg'},
 {'id': 42392,
  'name': 'Приключение',
  'slug': 'prikliuchenie',
  'language': 'rus',
  'games_count': 39061,
  'image_background': 'https://media.rawg.io/media/games/d82/d82990b9c67ba0d2d09d4e6fa88885a7.jpg'},
 {'id': 42398,
  'name': 'Инди',
  'slug': 'indi-2',
  'language': 'rus',
  'games_count': 56430,
  'image_background': 'https://media.rawg.io/media/gam

In [407]:
df3

,slug,name,released,tba,rating,rating_top,ratings_count,reviews_text_count,added,metacritic,playtime,suggestions_count,updated,reviews_count,platform_names,games_counts,store_names,games_counts_stores,genre_names,games_counts_genres,tags_names,tags_games_counts,age_rating,genre,subgenre_1,subgenre_2,subgenre_3,subgenre_4,subgenre_5,subgenre_6,subgenre_7,subgenre_8,subgenre_9,subgenre_10
0,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,6858,62,21120,92.0,74,434,2024-08-29T07:36:57,6964,"[PC, PlayStation 5, Xbox Series S/X, PlayStati...","[535023, 1107, 972, 6825, 3166, 2806, 5638]","[PlayStation Store, Epic Games, Steam, Xbox 36...","[7961, 1345, 98082, 1916, 4865]",['Action'],[181902],"[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 34016, 3406, 216...",Mature,'Action',None,None,None,None,None,None,None,None,None,None
1,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.65,5,6606,75,20603,92.0,45,677,2024-08-28T20:12:31,6715,"[Xbox Series S/X, PlayStation 5, macOS, PlaySt...","[972, 1107, 104581, 6825, 5503, 535023, 5638]","[GOG, PlayStation Store, Steam, Xbox Store, Ni...","[6282, 7961, 98082, 4865, 9046]","['Action', 'RPG']","[181902, 56940]","[Singleplayer, Full controller support, Atmosp...","[226836, 18689, 34016, 3406, 21690, 22242, 770...",Mature,'Action',RPG,None,None,None,None,None,None,None,None,None
2,portal-2,Portal 2,2011-04-18,False,4.60,5,5685,35,19487,95.0,11,553,2024-08-29T07:23:05,5737,"[PlayStation 3, PC, Xbox 360, Linux, macOS, Xb...","[3166, 535023, 2806, 77469, 104581, 5638]","[Xbox Store, Steam, PlayStation Store, Xbox 36...","[4865, 98082, 7961, 1916]","['Shooter', 'Puzzle']","[59498, 97296]","[Singleplayer, Steam Achievements, Multiplayer...","[226836, 39362, 38545, 18689, 18699, 34016, 75...",Everyone 10+,'Shooter',Puzzle,None,None,None,None,None,None,None,None,None
3,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.56,4,3479,27,17173,81.0,64,589,2024-08-20T10:48:54,3515,"[PC, Linux, Xbox 360, PlayStation 3]","[535023, 77469, 2806, 3166]","[PlayStation Store, Steam, Xbox 360 Store]","[7961, 98082, 1916]",['Shooter'],[59498],"[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 7569, 11918, 5164, 32327...",Mature,'Shooter',None,None,None,None,None,None,None,None,None,None
4,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,3896,14,16781,86.0,10,651,2024-08-25T15:59:03,3925,"[PlayStation 4, PlayStation 3, Xbox 360, macOS...","[6825, 3166, 2806, 104581, 535023, 5638]","[Xbox 360 Store, Steam, PlayStation Store, Goo...","[1916, 98082, 7961, 17097, 75559, 1345]","['Action', 'Fighting']","[181902, 11756]","[Singleplayer, Multiplayer, Full controller su...","[226836, 38545, 18689, 34016, 21690, 11887, 23...",Mature,'Action',Fighting,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31058,orbit,ORBIT,2011-12-06,False,0.00,0,0,0,18,73.0,1,248,2023-11-04T01:00:08,0,"[PC, Xbox One, PlayStation 3, PSP, Wii U]","[535023, 5638, 3166, 1371, 1126]","[Xbox Store, Steam, PlayStation Store, Nintend...","[4865, 98082, 7961, 9046]","['Action', 'Indie', 'Puzzle']","[181902, 68528, 97296]","[Steam Achievements, Multiplayer, Full control...","[39362, 38545, 18689, 32327, 196856, 11975, 68...",Everyone,'Action',Indie,Puzzle,None,None,None,None,None,None,None,None
31059,atari-flashback-classics-vol-1,Atari Flashback Classics Vol. 1,2016-10-13,False,0.00,0,3,0,18,73.0,0,103,2022-09-09T08:34:23,3,"[Xbox One, PlayStation 4]","[5638, 6825]","[PlayStation Store, Xbox Store]","[7961, 4865]",['Arcade'],[22658],"[Multiplayer, online, console, history, bundle...","[38545, 6557, 1390, 2374, 478, 1157]",Everyone,'Arcade',None,None,None,None,None,None,None,None,None,None
31067,the-casting-of-frank-stone,The Casting of Frank Stone,2024-09-03,False,0.00,0,0,0,17,73.0,0,297,2024-07-17

In [165]:
def extract_first_tag_info(tags):
    if isinstance(tags, list) and len(tags) > 0:
        first_tag = tags[0]
        return first_tag['id'], first_tag['name']
    return None, None 


In [166]:
df_clean1['first_tag_id'], df_clean1['first_tag_name'] = zip(*df_clean1['tags'].apply(extract_first_tag_info))


In [168]:
df_clean1[['first_tag_id', 'first_tag_name']]


,first_tag_id,first_tag_name
0,31.0,Singleplayer
1,31.0,Singleplayer
2,31.0,Singleplayer
3,31.0,Singleplayer
4,31.0,Singleplayer
...,...,...
1103,NaN,None
1104,NaN,None
1105,NaN,None
1106,NaN,None


In [ ]:
df_clean1.duplicated().sum()

TypeError: unhashable type: 'list'

In [ ]:
df_clean1['game_name'].value_counts()

game_name
Grand Theft Auto V                            1
Nickelodeon Kart Racers 3: Slime Speedway     1
Lost At Sea                                   1
I.Q.: Intelligent Qube                        1
A YEAR OF SPRINGS                             1
                                             ..
Destroy All Humans! 2 - Reprobed              1
Stranger of Paradise: Final Fantasy Origin    1
Asterigos: Curse of the Stars                 1
Beholder 3                                    1
Steel Seed                                    1
Name: count, Length: 1108, dtype: int64

In [214]:
df3['tags_names'][0]

['Singleplayer',
 'Steam Achievements',
 'Multiplayer',
 'Full controller support',
 'Atmospheric',
 'Great Soundtrack',
 'RPG',
 'Co-op',
 'Open World',
 'cooperative',
 'First-Person',
 'Third Person',
 'Funny',
 'Sandbox',
 'Comedy',
 'Third-Person Shooter',
 'Moddable',
 'Crime',
 'vr mod']

In [ ]:
{'App Store',
 'Epic Games',
 'GOG',
 'Nintendo Store',
 'PlayStation Store',
 'Steam',
 'Xbox 360 Store',
 'Xbox Store',
 'itch.io'}

{'App Store',
 'Epic Games',
 'GOG',
 'Nintendo Store',
 'PlayStation Store',
 'Steam',
 'Xbox 360 Store',
 'Xbox Store',
 'itch.io'}

In [ ]:
plataformas_columnas = ['SEGA Saturn', 'Xbox 360', 'Android', 'PlayStation 5', 'Game Boy Advance', 'Game Gear', 
                         'Nintendo 64', 'Game Boy', 'GameCube', 'PlayStation 3', 'SEGA Master System', 'Wii U', 
                         'Wii', 'iOS', 'SNES', 'Commodore / Amiga', 'Genesis', 'Nintendo Switch', 'Web', 
                         'Classic Macintosh', 'Nintendo 3DS', 'PS Vita', 'PSP', 'Nintendo DS', 'Nintendo DSi', 
                         'Dreamcast', 'Xbox One', 'PC', 'Xbox Series S/X', 'macOS', 'Xbox', 'PlayStation 2', 
                         'Linux', 'PlayStation 4', 'PlayStation']

df_clean1['numero_plataformas'] = df_clean1[plataformas_columnas].sum(axis=1)


In [ ]:
df_clean1['numero_plataformas'].value_counts()

numero_plataformas
6     312
5     245
3     227
4     118
2      68
7      59
8      34
1      33
9       6
10      3
13      1
12      1
22      1
Name: count, dtype: int64

In [ ]:
df_clean1.drop(columns=plataformas_columnas, inplace=True)

In [ ]:
nuevo_orden = ['game_id', 'game_name', 'released', 'tba', 'age rating', 'genre', 'subgenre', 'yet', 'owned', 'beaten', 'toplay',	'dropped', 'playing', 'numero_plataformas', 'rating_top', 'ratings_count', 'added', 'metacritic', 'suggestions_count', 'App Store', 'Epic Games', 'GOG','Nintendo Store','PlayStation Store','Steam','Xbox 360 Store','Xbox Store','itch.io']

In [ ]:
df_clean1 = df_clean1[nuevo_orden]

In [ ]:
df_clean1.isna().sum()

game_id                 0
game_name               0
released              105
tba                     0
age rating            552
genre                   0
subgenre              255
yet                   212
owned                  91
beaten                304
toplay                130
dropped               314
playing               399
numero_plataformas      0
rating_top              0
ratings_count           0
added                   0
metacritic            796
suggestions_count       0
App Store               0
Epic Games              0
GOG                     0
Nintendo Store          0
PlayStation Store       0
Steam                   0
Xbox 360 Store          0
Xbox Store              0
itch.io                 0
dtype: int64

In [ ]:
df_clean1['age rating'].value_counts()

age rating
Mature            200
Teen              165
Everyone 10+       70
Everyone           66
Rating Pending     30
Adults Only        25
Name: count, dtype: int64

In [ ]:
df_clean1['age rating']=df_clean1['age rating'].fillna('Mature')

/var/folders/zc/6rc7nzhj13scy5qssxw2pjlh0000gn/T/ipykernel_11978/975480331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean1['age rating']=df_clean1['age rating'].fillna('Mature')


In [ ]:
df_clean1.isna().sum()

game_id                 0
game_name               0
released              105
tba                     0
age rating              0
genre                   0
subgenre              255
yet                   212
owned                  91
beaten                304
toplay                130
dropped               314
playing               399
numero_plataformas      0
rating_top              0
ratings_count           0
added                   0
metacritic            796
suggestions_count       0
App Store               0
Epic Games              0
GOG                     0
Nintendo Store          0
PlayStation Store       0
Steam                   0
Xbox 360 Store          0
Xbox Store              0
itch.io                 0
dtype: int64

In [ ]:
#df_clean1.drop(columns='age_rating',inplace=True)

In [ ]:
df_clean1['subgenre'] = df_clean1['subgenre'].fillna('no')

/var/folders/zc/6rc7nzhj13scy5qssxw2pjlh0000gn/T/ipykernel_11978/339338487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean1['subgenre'] = df_clean1['subgenre'].fillna('no')


In [ ]:
df_clean1['metacritic'] = df_clean1['metacritic'].fillna(0)

/var/folders/zc/6rc7nzhj13scy5qssxw2pjlh0000gn/T/ipykernel_11978/3720782326.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean1['metacritic'] = df_clean1['metacritic'].fillna(0)


In [ ]:
df_clean1.isna().sum()

game_id                 0
game_name               0
released              105
tba                     0
age rating              0
genre                   0
subgenre                0
yet                   212
owned                  91
beaten                304
toplay                130
dropped               314
playing               399
numero_plataformas      0
rating_top              0
ratings_count           0
added                   0
metacritic              0
suggestions_count       0
App Store               0
Epic Games              0
GOG                     0
Nintendo Store          0
PlayStation Store       0
Steam                   0
Xbox 360 Store          0
Xbox Store              0
itch.io                 0
dtype: int64

In [408]:
df3['released'] = pd.to_datetime(df3['released'])

In [409]:
df3.dtypes

slug                           object
name                           object
released               datetime64[ns]
tba                              bool
rating                        float64
rating_top                      int64
ratings_count                   int64
reviews_text_count              int64
added                           int64
metacritic                    float64
playtime                        int64
suggestions_count               int64
updated                        object
reviews_count                   int64
platform_names                 object
games_counts                   object
store_names                    object
games_counts_stores            object
genre_names                    object
games_counts_genres            object
tags_names                     object
tags_games_counts              object
age_rating                     object
genre                          object
subgenre_1                     object
subgenre_2                     object
subgenre_3  

In [410]:
df3['updated'] = pd.to_datetime(df3['updated'])


In [ ]:
df_clean1 = df_clean1[df_clean1['tba'] != True]

In [ ]:
df_clean1.isna().sum()

game_id                 0
game_name               0
released                4
tba                     0
age rating              0
genre                   0
subgenre                0
yet                   164
owned                  59
beaten                203
toplay                120
dropped               208
playing               305
numero_plataformas      0
rating_top              0
ratings_count           0
added                   0
metacritic              0
suggestions_count       0
App Store               0
Epic Games              0
GOG                     0
Nintendo Store          0
PlayStation Store       0
Steam                   0
Xbox 360 Store          0
Xbox Store              0
itch.io                 0
dtype: int64

In [ ]:
df_date = df_clean1[df_clean1['released'].isna()]


In [ ]:
drop=df_date.index
df_clean1.drop(drop,inplace=True)

In [ ]:
df_clean1.drop(columns=['yet', 'beaten', 'toplay', 'dropped', 'playing' ],inplace=True)

In [ ]:
df_clean1.isna().sum()

game_id                0
game_name              0
released               0
tba                    0
age rating             0
genre                  0
subgenre               0
owned                 59
numero_plataformas     0
rating_top             0
ratings_count          0
added                  0
metacritic             0
suggestions_count      0
App Store              0
Epic Games             0
GOG                    0
Nintendo Store         0
PlayStation Store      0
Steam                  0
Xbox 360 Store         0
Xbox Store             0
itch.io                0
dtype: int64

In [ ]:
df_clean1['owned'].mean()

472.5108695652174

In [ ]:
df_clean1['owned']=df_clean1['owned'].fillna(472)

In [ ]:
df_clean1

,game_id,game_name,released,tba,age rating,genre,subgenre,owned,numero_plataformas,rating_top,ratings_count,added,metacritic,suggestions_count,App Store,Epic Games,GOG,Nintendo Store,PlayStation Store,Steam,Xbox 360 Store,Xbox Store,itch.io
0,3498,Grand Theft Auto V,2013-09-17,False,Mature,Action,no,12151.0,7,5,6855,21102,92.0,434,False,True,False,False,True,True,True,True,False
1,3328,The Witcher 3: Wild Hunt,2015-05-18,False,Mature,Action,RPG,12006.0,7,5,6599,20587,92.0,677,False,False,True,True,True,True,False,True,False
2,5679,The Elder Scrolls V: Skyrim,2011-11-11,False,Mature,Action,RPG,9519.0,8,5,4632,15890,94.0,602,False,False,False,True,True,True,True,False,False
3,32,Destiny 2,2017-09-06,False,Teen,Shooter,Action,10458.0,6,4,2594,13773,82.0,1230,False,True,False,False,True,True,False,True,False
4,3070,Fallout 4,2015-11-09,False,Mature,Action,RPG,8544.0,4,4,3273,13481,84.0,460,False,False,False,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,795323,Dead by Daylight - Hellraiser,2021-09-07,False,Mature,Action,no,472.0,6,0,0,0,0.0,394,False,False,False,False,True,True,False,False,False
1101,812169,SENSEs: Midnight,2022-07-22,False,Teen,Adventure,no,472.0,7,0,0,0,0.0,371,False,False,False,True,True,True,False,True,False
1103,795327,Dead by Daylight - Spark of Madness,2017-05-11,False,Mature,Adventure,Action,472.0,6,0,0,0,0.0,384,False,False,False,False,False,True,False,False,False
1104,887670,Pre-purchase Romancing SaGa - Minstrel Song- Remastered,2022-12-01,False,Mature,RPG,no,472.0,4,0,0,0,0.0,162,False,False,False,True,False,True,False,False,False


In [ ]:
df_clean1.isna().sum()

game_id               0
game_name             0
released              0
tba                   0
age rating            0
genre                 0
subgenre              0
owned                 0
numero_plataformas    0
rating_top            0
ratings_count         0
added                 0
metacritic            0
suggestions_count     0
App Store             0
Epic Games            0
GOG                   0
Nintendo Store        0
PlayStation Store     0
Steam                 0
Xbox 360 Store        0
Xbox Store            0
itch.io               0
dtype: int64

In [ ]:
df_clean1

,game_id,game_name,released,tba,age rating,genre,subgenre,owned,numero_plataformas,rating_top,ratings_count,added,metacritic,suggestions_count,App Store,Epic Games,GOG,Nintendo Store,PlayStation Store,Steam,Xbox 360 Store,Xbox Store,itch.io
0,3498,Grand Theft Auto V,2013-09-17,False,Mature,Action,no,12151.0,7,5,6855,21102,92.0,434,False,True,False,False,True,True,True,True,False
1,3328,The Witcher 3: Wild Hunt,2015-05-18,False,Mature,Action,RPG,12006.0,7,5,6599,20587,92.0,677,False,False,True,True,True,True,False,True,False
2,5679,The Elder Scrolls V: Skyrim,2011-11-11,False,Mature,Action,RPG,9519.0,8,5,4632,15890,94.0,602,False,False,False,True,True,True,True,False,False
3,32,Destiny 2,2017-09-06,False,Teen,Shooter,Action,10458.0,6,4,2594,13773,82.0,1230,False,True,False,False,True,True,False,True,False
4,3070,Fallout 4,2015-11-09,False,Mature,Action,RPG,8544.0,4,4,3273,13481,84.0,460,False,False,False,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,795323,Dead by Daylight - Hellraiser,2021-09-07,False,Mature,Action,no,472.0,6,0,0,0,0.0,394,False,False,False,False,True,True,False,False,False
1101,812169,SENSEs: Midnight,2022-07-22,False,Teen,Adventure,no,472.0,7,0,0,0,0.0,371,False,False,False,True,True,True,False,True,False
1103,795327,Dead by Daylight - Spark of Madness,2017-05-11,False,Mature,Adventure,Action,472.0,6,0,0,0,0.0,384,False,False,False,False,False,True,False,False,False
1104,887670,Pre-purchase Romancing SaGa - Minstrel Song- Remastered,2022-12-01,False,Mature,RPG,no,472.0,4,0,0,0,0.0,162,False,False,False,True,False,True,False,False,False


In [ ]:
df_clean1['numero_stores'] = df_clean1[['App Store', 'Epic Games', 'GOG', 'Nintendo Store', 
                                        'PlayStation Store', 'Steam', 'Xbox 360 Store', 
                                        'Xbox Store', 'itch.io']].sum(axis=1)

In [ ]:
stores_drop = ['App Store', 'Epic Games', 'GOG', 'Nintendo Store', 
                                        'PlayStation Store', 'Steam', 'Xbox 360 Store', 
                                        'Xbox Store', 'itch.io']

In [ ]:
df_clean1 = df_clean1.drop(columns=stores_drop)

In [ ]:
df_clean1.isna().sum()

game_id               0
game_name             0
released              0
tba                   0
age rating            0
genre                 0
subgenre              0
owned                 0
numero_plataformas    0
rating_top            0
ratings_count         0
added                 0
metacritic            0
suggestions_count     0
numero_stores         0
dtype: int64

In [ ]:
df_clean1

,game_id,game_name,released,tba,age rating,genre,subgenre,owned,numero_plataformas,rating_top,ratings_count,added,metacritic,suggestions_count,numero_stores
0,3498,Grand Theft Auto V,2013-09-17,False,Mature,Action,no,12151.0,7,5,6855,21102,92.0,434,5
1,3328,The Witcher 3: Wild Hunt,2015-05-18,False,Mature,Action,RPG,12006.0,7,5,6599,20587,92.0,677,5
2,5679,The Elder Scrolls V: Skyrim,2011-11-11,False,Mature,Action,RPG,9519.0,8,5,4632,15890,94.0,602,4
3,32,Destiny 2,2017-09-06,False,Teen,Shooter,Action,10458.0,6,4,2594,13773,82.0,1230,4
4,3070,Fallout 4,2015-11-09,False,Mature,Action,RPG,8544.0,4,4,3273,13481,84.0,460,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,795323,Dead by Daylight - Hellraiser,2021-09-07,False,Mature,Action,no,472.0,6,0,0,0,0.0,394,2
1101,812169,SENSEs: Midnight,2022-07-22,False,Teen,Adventure,no,472.0,7,0,0,0,0.0,371,4
1103,795327,Dead by Daylight - Spark of Madness,2017-05-11,False,Mature,Adventure,Action,472.0,6,0,0,0,0.0,384,1
1104,887670,Pre-purchase Romancing SaGa - Minstrel Song- Remastered,2022-12-01,False,Mature,RPG,no,472.0,4,0,0,0,0.0,162,2


In [ ]:
#df_clean1.to_csv('df_clean_final.csv')